## 1 Instalação SPARQLWrapper e Deep_Translator

In [1]:
!pip install SPARQLWrapper deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 9.9 MB/s 
     |████████████████████████████████| 41 kB 611 kB/s 
     |████████████████████████████████| 128 kB 65.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


# 2 Import das bibliotecas

In [2]:
# importando biblioteca responsável por fazer consultas em SPARQL na DBpedia
from SPARQLWrapper import SPARQLWrapper, JSON
from deep_translator import GoogleTranslator
import re

sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# 3 Definição dos tipos a serem observados

## 3.1 Classificação dos tipos retornados pela DBpedia

1. CHEM
 - Drug
 - Chemical Substance
 - Chemical Compound

2. DISO
 - Disease

3. ANAT
  - Anatomical Structure

4. PROC
  - Work

5. SPEC
  - Medical Specialty
  - Person Function

6. FOOD
  - Food

In [3]:
from enum import Enum

class SG(Enum):
  CHEM = 1
  DISO = 2
  ANAT = 3
  PROC = 4
  SPEC = 5
  FOOD = 6

# Dicionário de tipos correspondentes aos grupos semanticos observados
TIPOS_OBSERVADOS = {
  'Drug' : 1,
  'ChemicalSubstance': 1,
  'ChemicalCompound': 1,
  'Disease' : 2,
  'AnatomicalStructure' : 3,
  'Work': 4,
  'MedicalSpecialty' : 5,
  'PersonFunction' : 5,
  'Food' : 6
}

## 3.2 Funções de consulta ao DBpedia

In [4]:
# Dicionário onde serão guardados os tipos retornados pela DBpedia de todass as palavras
TODOS_OS_TIPOS = dict()

In [5]:
def getListaTiposDBpedia(sentenca):
  """
    Função responsável por consultar cada palavra da sentença enviada como parâmetro
    na base de ontologias da DBpedia e verificar se os tipos de entidade retornados
    estão dentro dos tipos a serem observados

    sentença {String} -- sentença com as palavras a serem consultadas
  """
  
  # remove parenteses e outros caracteres especiais
  sentenca = re.sub(u"[^\w\- ]", '', sentenca)
  
  # apenas palavras com tamanho maior que 2 e que não são apenas dígitos
  palavras = [x.capitalize() for x in sentenca.split() if (len(x) > 2 and not x.isdigit())]

  for palavra in palavras:
    
    # remove hifens nas estremidades das palavras
    palavra = re.sub(r"^\W|\W\Z", "", palavra)
    
    # Coloca a palavra no dicionário para não fazer uma segunda consulta com a mesma palavra
    if not palavra in TODOS_OS_TIPOS:
      TODOS_OS_TIPOS[palavra] = []

      # Query usada para fazer a pesquisa na DBPedia
      """
      select distinct ?i, ?type
        where {{ ?i rdfs:label "Cetoprofeno"@pt ; 
        a ?type . 
        FILTER (strstarts(str(?type), str("http://dbpedia.org/ontology/"))) 
        FILTER (strstarts(str(?i), str("http://dbpedia.org/resource/Ketoprofen"))) --palavra em ingles. 
      }}
      """
      query = f'select distinct ?type where {{ ?i rdfs:label "{palavra}"@pt ; a ?type . FILTER (strstarts(str(?type), str("http://dbpedia.org/ontology/"))) }}'
      
      sparql.setQuery(query)
      
      # Configura o retorno para o formato JSON
      sparql.setReturnFormat(JSON)
      
      # Realiza a consulta
      dbpedia_return_dict = sparql.query().convert()
      
      # Obtem a lista de "types" retornados pela consulta
      return_list = dbpedia_return_dict['results']['bindings']

      # Verifica se existem resultados
      if return_list:
        
        tipos_dbpedia_retornados = [result['type']['value'].split('/')[-1] for result in return_list]

        tipos_obsdervados_encontrados = [tipo for tipo in tipos_dbpedia_retornados if tipo in TIPOS_OBSERVADOS]

        tipos_codificados = []
        for tipo in tipos_obsdervados_encontrados:
          if TIPOS_OBSERVADOS[tipo] not in tipos_codificados:
            tipos_codificados.append(TIPOS_OBSERVADOS[tipo])
        
        # Se foi identificado apenas 1 tipo observado no retorno
        if len(tipos_codificados) == 1:
          
          TODOS_OS_TIPOS[palavra] = tipos_codificados
          print('Palavra: ' + palavra + ' tipo adicionado: ' + str(tipos_codificados[0]))

        # Caso a consulta tenha retornado mais de 1 dos tipos observados
        # faz uma nova verificação pelo usando mais 1 filtro para a palavra em ingles
        elif len(tipos_codificados) > 1:

          palavra_em_ingles = GoogleTranslator(source='pt', target='en').translate(palavra).capitalize()
          # print(f'palavra: {palavra}  tradução: {palavra_em_ingles}')
          
          query = f'select distinct ?type where {{ ?i rdfs:label "{palavra}"@pt ; a ?type . FILTER (strstarts(str(?type), str("http://dbpedia.org/ontology/"))) FILTER (strstarts(str(?i), str("http://dbpedia.org/resource/{palavra_em_ingles}"))) }}'
      
          sparql.setQuery(query)
          
          # Configura o retorno para o formato JSON
          sparql.setReturnFormat(JSON)
          
          # Realiza a consulta
          dbpedia_return_dict = sparql.query().convert()
          
          # Obtem a lista de "types" retornados pela consulta com filtro da palavra em ingles
          return_list_english = dbpedia_return_dict['results']['bindings']

          if return_list_english:
        
            tipos_dbpedia_retornados_ingles = [result['type']['value'].split('/')[-1] for result in return_list_english]

            tipos_obsdervados_encontrados_ingles = [tipo for tipo in tipos_dbpedia_retornados_ingles if tipo in TIPOS_OBSERVADOS]

            tipos_codificados_ingles = []
            # Adiciona na lista apenas 1 valor de cada tipo observado
            for tipo in tipos_obsdervados_encontrados_ingles:
              if TIPOS_OBSERVADOS[tipo] not in tipos_codificados_ingles:
                tipos_codificados_ingles.append(TIPOS_OBSERVADOS[tipo])
        
            if len(tipos_codificados_ingles) == 1:
              TODOS_OS_TIPOS[palavra] = tipos_codificados_ingles
              print('Palavra: ' + palavra + ' tipo adicionado: ' + str(tipos_codificados_ingles[0]))

            else:
              TODOS_OS_TIPOS[palavra] = tipos_codificados
              print('Palavra: ' + palavra + ' tipo adicionado: ') 
              print(tipos_codificados)

      else:
        # Faz uma segunda busca pela palavra em maiúscula
        palavra_upper = palavra.upper()

        # Query usada para fazer a pesquisa na DBPedia
        query = f'select distinct ?type where {{ ?i rdfs:label "{palavra_upper}"@pt ; a ?type . FILTER (strstarts(str(?type), str("http://dbpedia.org/ontology/"))) }}'
        sparql.setQuery(query)
        
        # Configura o retorno para o formato JSON
        sparql.setReturnFormat(JSON)
        
        # Realiza a consulta
        dbpedia_return_dict = sparql.query().convert()
        
        # Obtem a lista de "types" retornados pela consulta
        return_list_upper = dbpedia_return_dict['results']['bindings']

        # Verifica se existem resultados
        if return_list_upper:
          TODOS_OS_TIPOS[palavra_upper] = []

          tipos_dbpedia_retornados = [result['type']['value'].split('/')[-1] for result in return_list]

          tipos_obsdervados_encontrados = [tipo for tipo in tipos_dbpedia_retornados if tipo in TIPOS_OBSERVADOS]

          tipos_codificados = []
          for tipo in tipos_obsdervados_encontrados:
            if TIPOS_OBSERVADOS[tipo] not in tipos_codificados:
              tipos_codificados.append(TIPOS_OBSERVADOS[tipo])

          if len(tipos_codificados) == 1:
            TODOS_OS_TIPOS[palavra_upper] = tipos_codificados
            print('Palavra: ' + palavra_upper + ' tipo adicionado: ' + str(tipos_codificados[0]))

          # Caso a consulta tenha retornado mais de 1 dos tipos observados
          # faz uma nova verificação pelo usando mais 1 filtro para a palavra em ingles
          elif len(tipos_codificados) > 1:
            palavra_em_ingles = GoogleTranslator(source='pt', target='en').translate(palavra).capitalize()
            # print(f'palavra: {palavra}  tradução: {palavra_em_ingles}')

            query = f'select distinct ?type where {{ ?i rdfs:label "{palavra_upper}"@pt ; a ?type . FILTER (strstarts(str(?type), str("http://dbpedia.org/ontology/"))) FILTER (strstarts(str(?i), str("http://dbpedia.org/resource/{palavra_em_ingles}"))) }}'
      
            sparql.setQuery(query)
          
            # Configura o retorno para o formato JSON
            sparql.setReturnFormat(JSON)
            
            # Realiza a consulta
            dbpedia_return_dict = sparql.query().convert()
            
            # Obtem a lista de "types" retornados pela consulta com filtro da palavra em ingles
            return_list_english = dbpedia_return_dict['results']['bindings']

            if return_list_english:
          
              tipos_dbpedia_retornados_ingles = [result['type']['value'].split('/')[-1] for result in return_list_english]

              tipos_obsdervados_encontrados_ingles = [tipo for tipo in tipos_dbpedia_retornados_ingles if tipo in TIPOS_OBSERVADOS]

              tipos_codificados_ingles = []
              for tipo in tipos_obsdervados_encontrados_ingles:
                if TIPOS_OBSERVADOS[tipo] not in tipos_codificados_ingles:
                  tipos_codificados_ingles.append(TIPOS_OBSERVADOS[tipo])
          
              if len(tipos_codificados_ingles) == 1:
                TODOS_OS_TIPOS[palavra_upper] = tipos_codificados_ingles
                print('Palavra: ' + palavra + ' tipo adicionado: ' + str(tipos_codificados_ingles[0]))

              else:
                TODOS_OS_TIPOS[palavra_upper] = tipos_codificados
                print('Palavra: ' + palavra_upper + ' tipo adicionado: ')
                print(tipos_codificados)


# 4 Gerar NER

In [10]:
# Exemplo de documentos a serem consultados:
lista_documentos = []
lista_documentos.append(
    "Prescrevo Cetoprofeno e descolonização da pele com Triclosan,\
    e manter Mometasona; Oriento buscar atendimento com Dermatologista caso não haja melhora;")

In [11]:
# Import das bibliotecas.
import os # Biblioteca para acessar o sistema de arquivos
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso

# Percorre as linhas dos documentos
for sentenca in lista_documentos:

  # Carrega os tipos das palavras da sentença no DBpedia
  getListaTiposDBpedia(sentenca)


Palavra: Cetoprofeno tipo adicionado: 1
Palavra: Pele tipo adicionado: 3
Palavra: Triclosan tipo adicionado: 1
Palavra: Mometasona tipo adicionado: 1


In [23]:
# Dicionário que recebe os resultados
for palavra in TODOS_OS_TIPOS.keys():
  tipo = TODOS_OS_TIPOS[palavra]
  print(f'Palavra: {palavra}, Tipo: {tipo}')

Palavra: Prescrevo, Tipo: []
Palavra: Cetoprofeno, Tipo: [1]
Palavra: Descolonização, Tipo: []
Palavra: Pele, Tipo: [3]
Palavra: Com, Tipo: []
Palavra: Triclosan, Tipo: [1]
Palavra: Manter, Tipo: []
Palavra: Mometasona, Tipo: [1]
Palavra: Oriento, Tipo: []
Palavra: Buscar, Tipo: []
Palavra: Atendimento, Tipo: []
Palavra: Dermatologista, Tipo: []
Palavra: Caso, Tipo: []
Palavra: Não, Tipo: []
Palavra: Haja, Tipo: []
Palavra: Melhora, Tipo: []
